In [ ]:
## !pip install -U tensorflow==2.7.1
## !pip install scikeras
!pip install pretty-confusion-matrix
!pip install transformers
!pip install pytorch-transformers
!pip install -U sentence-transformers

In [ ]:
import numpy as np 
import pandas as pd 
import csv
import pickle
import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf
from keras.layers import Bidirectional, Input, Dense, Layer, Dropout, LSTM, RepeatVector, Concatenate, Flatten
from keras.models import Sequential, Model
from tensorflow.python.keras.callbacks import EarlyStopping
from keras import backend as K

In [ ]:
print(tf.__version__)

In [ ]:
NUM_OF_ASPECTS = 7
EPOCHS = 150
MAXLEN = 768
# Is_BiLSTM_Present = True
LSTM_UNITS = 256
SAVE_PATH = "./Without-BiLSTM-epoch150.h5"

## **1) DATASET** : 

I. **REVIEWS** <br>
II. **LABELS:**<br>
1. Clarity (CLA)
2. Motivation/Impact (MOT)
2.   Originality (ORI)
3.   Soundness/Correctness (SOU)
2.   Substance (SUB)
1.   Replicability (REP)
2.   Meaningful Comparison (CMP)


In [ ]:
# LOAD DATASET
# df = pd.read_csv('../input/processed-aspect/upsampled_ohe_aspect_GoldStandard.csv' )
# df.dropna(inplace=True)
# df


In [ ]:
# X = df.review
# y = df[['MOT','ORI','SOU','SUB','REP','CMP','CLA']]



---




## **2) EMBEDDINGS**

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModel

In [ ]:
## BERT BASE
# embed_model_name = 'bert-base-uncased'

# BIO-BERT
# embed_model_name = 'gsarti/biobert-nli'

## SCIBERT
## embed_model_name = 'allenai/scibert_scivocab_uncased'
embed_model_name = 'gsarti/scibert-nli'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(embed_model_name) 
embed_model = AutoModel.from_pretrained(embed_model_name)

In [ ]:
def sent2embed(model, sent):
    inputs = tokenizer(sent, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state
    return outputs

# SAMPLE SCIBERT
sent = ['This work studies the predictive uncertainty issue of deep learning models.']
outputs = sent2embed(embed_model, sent)
outputs[1].shape


In [ ]:
# PRE-PROCESS -- SCIBERT-EMBED
def sciBertEmbed(reviews, y_train, model):
    embed_arr = []
    #   y_train_arr=[]
    start = reviews.index[0]
    for idx in range(len(reviews.index)):
        print(start + idx)
        outputs = sent2embed(model, reviews.loc[start+idx])
        embed_arr.append([outputs[1].detach().numpy()[0]])
        # y_train_arr.append([y_train.loc[0].values])

    embed_train_arr = tf.stack(embed_arr)
    # y_train = tf.stack(y_train_arr)

    return embed_train_arr, y_train
  

# sciBertEmbed(X[1], y[1])

In [ ]:
# embeds, y_ohe = sciBertEmbed(X.loc[:10], y.loc[:10], embed_model)

In [ ]:
# embeds.shape , y_ohe.shape



---





In [ ]:
# LOAD EMBEDS DATASET
import pickle

# aspect_categories = ['CLA','CMP','MOT','ORI','REP','SOU','SUB'] 
# polarity_labels = ['POL_NEG', 'POL_NEU', 'POL_POS']

# with open('../input/multitaskaspectdataset/train_embeds.pickle', 'rb') as handle:
#     train_embeds = pickle.load(handle)

# with open('../input/multitaskaspectdataset/y_train.pickle', 'rb') as handle:
#     y_train = pickle.load(handle)
 
# with open('../input/multitaskaspectdataset/test_embeds.pickle', 'rb') as handle:
#     test_embeds = pickle.load(handle)

# with open('../input/multitaskaspectdataset/y_test.pickle', 'rb') as handle:
#     y_test = pickle.load(handle)


# LOAD REVIEWS LABELS & EMBEDS
with open('../input/multitask-pos-scibert/reviews_train.pickle', 'rb') as handle:
    reviews_train = pickle.load(handle)
    handle.close()

with open('../input/multitask-pos-scibert/reviews_test.pickle', 'rb') as handle:
    reviews_test = pickle.load(handle)
    handle.close()

with open('../input/multitask-pos-scibert/y_ohe_train.pickle', 'rb') as handle:
    y_ohe_train = pickle.load(handle)
    handle.close()
    
with open('../input/multitask-pos-scibert/y_ohe_test.pickle', 'rb') as handle:
    y_ohe_test = pickle.load(handle)
    handle.close()
    
with open('../input/multitask-pos-scibert/pos_embeds_train.pickle', 'rb') as handle:
    pos_embeds_train = pickle.load(handle)
    handle.close() 
    
with open('../input/multitask-pos-scibert/pos_embeds_test.pickle', 'rb') as handle:
    pos_embeds_test = pickle.load(handle)
    handle.close() 

# SCIBERT
with open('../input/multitask-pos-scibert/sci_embeds_train.pickle', 'rb') as handle:
    sci_embeds_train = pickle.load(handle)
    handle.close()
    
with open('../input/multitask-pos-scibert/sci_embeds_test.pickle', 'rb') as handle:
    sci_embeds_test = pickle.load(handle)
    handle.close()
    
# BERT-BASE
with open('../input/multitask-pos-scibert/bert_embeds_train.pickle', 'rb') as handle:
    bert_embeds_train = pickle.load(handle)
    handle.close()
    
with open('../input/multitask-pos-scibert/bert_embeds_test.pickle', 'rb') as handle:
    bert_embeds_test = pickle.load(handle)
    handle.close()
    
# BIO-BERT
with open('../input/multitask-pos-scibert/bioBert_embeds_train.pickle', 'rb') as handle:
    bio_embeds_train = pickle.load(handle)
    handle.close()
    
with open('../input/multitask-pos-scibert/bioBert_embeds_test.pickle', 'rb') as handle:
    bio_embeds_test = pickle.load(handle)
    handle.close()

In [ ]:
aspect_categories = y_ohe_train.columns[:-1]
y_ohe_train['polarity'] = np.where(y_ohe_train['polarity'] == -1, 0, 1)
y_ohe_test['polarity'] = np.where(y_ohe_test['polarity'] == -1, 0, 1)

In [ ]:
train_embeds = sci_embeds_train
test_embeds = sci_embeds_test

In [ ]:
train_embeds.shape , y_ohe_train.shape

In [ ]:
test_embeds.shape , y_ohe_test.shape

### **3) BiLSTM with attention**

In [ ]:
class Attention(Layer):
    
    def __init__(self, return_sequences=True, **kwargs):
        super(Attention,self).__init__()
        self.return_sequences = return_sequences
        super(Attention, self).__init__(**kwargs)

    def get_config(self):
        config = super(Attention, self).get_config().copy()
        config.update({
            'return_sequences': self.return_sequences , 
        })
        return config


    def build(self, input_shape):
        
        self.W=self.add_weight(name="att_weight", shape=(input_shape[-1],1),
                               initializer="normal")
        self.b=self.add_weight(name="att_bias", shape=(input_shape[1],1),
                               initializer="zeros")
        
        super(Attention,self).build(input_shape)
        
    def call(self, x):
        
        
#      TRY 1
        e =K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        a =K.softmax(e)
        a=K.expand_dims(a,axis=-1)
        
        
#         TRY 2
#         e = K.tanh(K.dot(x,self.W)+self.b)
    
        
#         METHOD OLD
#         e = K.dot(x,self.W) + self.b
   
#         a = K.softmax(e, axis=1)
        
        output = x*a
        
        return K.sum(output, axis=1) 

In [ ]:
## REDUCE DIMENSION OF DATASET BY 1DIM -- ONLY IF BiLSTM NOT there
train_embeds2 = np.array([row[0] for row in train_embeds ])
test_embeds2 = np.array([row[0] for row in test_embeds ])


# # # OTHERS
# train_embeds2 = train_embeds
# test_embeds2 = test_embeds 

In [ ]:
train_embeds2.shape , test_embeds2.shape

In [ ]:
# FUNCTIONAL API

# embed = Input(shape=  (None, MAXLEN), name='INPUT') # with BiLSTM
embed = Input(shape=  (MAXLEN,), name='INPUT') # without BiLSTM

# biLSTM = Bidirectional(LSTM(LSTM_UNITS , return_sequences=False) , name='BILSTM' )(embed)
repeat_op = RepeatVector(1)(embed)
# 
attentions=[]
for i in range(NUM_OF_ASPECTS):
  x = Attention(return_sequences=False)(repeat_op)
  attentions.append(x)

concate_attention  = Concatenate()(attentions)
flatten = Flatten()(concate_attention)

# fc = Dense(4*MAXLEN, activation='relu', name='FULLY_CONNECTED')(embed)
# fc = Dropout(0.2)(fc)

# BRANCH 1
b1_x =  Dense(1000, activation='relu', name='CATEGORY_DENSE')(flatten)
category_output = Dense(NUM_OF_ASPECTS, activation='softmax', name='CATEGORY_OUTPUT')(b1_x)
# b1_x =  Dense(2*MAXLEN, activation='relu', name='CATEGORY_DENSE')(fc)
# b1_x = Dropout(0.2)(b1_x)
# b1_x =  Dense(MAXLEN, activation='relu', name='CATEGORY_DENSE_1')(b1_x)
# b1_x = Dropout(0.2)(b1_x)
category_output = Dense(NUM_OF_ASPECTS, activation='softmax', name='CATEGORY_OUTPUT')(b1_x)

# BRANCH 2
b2_x =  Dense(60, activation='relu', name='POLARITY_DENSE')(flatten)
# b2_x =  Dense(2*MAXLEN, activation='relu', name='POLARITY_DENSE')(fc)
# b2_x = Dropout(0.2)(b2_x)
# b2_x =  Dense(MAXLEN, activation='relu', name='POLARITY_DENSE_1')(b2_x)
# b2_x = Dropout(0.2)(b2_x)
polarity_output = Dense(1, activation='sigmoid', name='POLARITY_OUTPUT')(b2_x)

multitask_model = Model(inputs=embed, outputs=[category_output, polarity_output])

In [ ]:
metrics =[ 
    'accuracy', 
    tf.keras.metrics.Precision(), 
    tf.keras.metrics.Recall()
]

# cb_early_stopper = EarlyStopping(monitor = 'val_loss', patience = 5)

loss = { 'CATEGORY_OUTPUT': 'categorical_crossentropy', 
        'POLARITY_OUTPUT': 'binary_crossentropy'
       }

lossWeights = { 'CATEGORY_OUTPUT': 1.0, 
               'POLARITY_OUTPUT': 1.0
              }


multitask_model.compile(loss= loss,
                        loss_weights = lossWeights,
                        optimizer='adam', 
                        metrics=['accuracy']) 


multitask_model.summary()


In [ ]:
tf.keras.utils.plot_model(multitask_model, to_file='multi-task_model.png', show_shapes=True)

In [ ]:
# EPOCHS = 150
history = multitask_model.fit(
                          train_embeds2, 
                          y =  {
                          'CATEGORY_OUTPUT': y_ohe_train[aspect_categories].values, 
                          'POLARITY_OUTPUT': y_ohe_train.polarity.values
                          },  
                           validation_data = (test_embeds2, {
                          'CATEGORY_OUTPUT': y_ohe_test[aspect_categories].values, 
                          'POLARITY_OUTPUT': y_ohe_test.polarity.values
                           }),
                           epochs = EPOCHS,
                           shuffle=True,
                           verbose=2
                          )

### CROSS VAL TRAINING

In [ ]:
# from scikeras.wrappers import KerasClassifier
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import cross_validate
# from sklearn.metrics import make_scorer
# from sklearn.metrics import accuracy_score, precision_score, recall_score
# from sklearn.model_selection import StratifiedKFold

# kfold = StratifiedKFold(n_splits=2, shuffle=True)
# cvscores = []
# for train, test in kfold.split(train_embeds, y_train):
#     multitask_model = Model(inputs=embed, outputs=[category_output, polarity_output])
#     history = multitask_model.fit(train_embeds.loc[train], 
#                                   [y_train[aspect_categories].loc[train], y_train.polarity].loc[train], 
#                                   epochs=EPOCHS, 
#                                   validation_split=0.15,
#                                   verbose=2)
    
#     scores = multitask_model.evaluate(test_embeds.loc[test], 
#                                       [y_test[aspect_categories], y_test.polarity], 
#                                       verbose=2)
#     print(scores)


In [ ]:
## BOTH DOESN'T WORK

# scorers = ['precision', 'recall', 'f1'] 
# estimator = KerasClassifier(model=multitask_model, epochs=EPOCHS, verbose=2)
# scores = cross_validate(estimator, train_embeds, [y_train[aspect_categories].values, y_train.polarity.values], 
#                         cv=kfold,
#                         scoring=scoring, 
#                         return_train_score=False)


# results = cross_val_score(estimator, train_embeds, 
#                           [y_train[aspect_categories].values, y_train.polarity.values], 
# #                           scoring=scorers,
#                           cv=kfold)

In [ ]:
multitask_model.save(SAVE_PATH, overwrite=True, save_format="tf")
print("Weights Saved")

In [ ]:
# LOAD MODEL
from keras.models import load_model
LOAD_PATH = SAVE_PATH
# LOAD_PATH = "../input/berth5/MultiTask-Ablation_WithoutAttention-epoch150.h5"
model = load_model(LOAD_PATH, custom_objects={'Attention': Attention})

print("MODEL LOADED\n\n")

model.evaluate(test_embeds2, [y_ohe_test[aspect_categories].values, y_ohe_test.polarity.values] )

_______________________________________________________________________________________________________________________________________

## ANALYSIS

In [ ]:
history.history.keys()

In [ ]:
metrics_keys = list(history.history.keys())

loss = history.history[metrics_keys[0]]
category_loss = history.history[metrics_keys[1]]
polarity_loss = history.history[metrics_keys[2]]
category_acc = history.history[metrics_keys[3]]
polarity_acc = history.history[metrics_keys[4]]
val_loss = history.history[metrics_keys[5]]
val_category_loss = history.history[metrics_keys[6]]
val_polarity_loss = history.history[metrics_keys[7]]
val_category_acc = history.history[metrics_keys[8]]
val_polarity_acc = history.history[metrics_keys[9]]


print("\nCATEGORY ACCURACY : " , category_acc)
print("\nPOLARITY ACCURACY : " , polarity_acc)
print("\nVAL CATEGORY ACCURACY : " ,val_category_acc)
print("\nVAL POLARITY ACCURACY : " ,val_polarity_acc)

print("\nLOSS : " , loss)
print("\nVAL LOSS : " , val_loss)
print("\nCATEGORY LOSS : " , category_loss)
print("\nCATEGORY VAL LOSS : " , val_category_loss)
print("\nPOLARITY LOSS : " , polarity_loss)
print("\nVAL POLARITY LOSS : " , val_polarity_loss)

In [ ]:
metrics = history.history['loss']
epochs_range = range(1, len(metrics) + 1) 

plt.figure(figsize=(23, 23))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, category_acc, label='Training Accuracy')
plt.plot(epochs_range, val_category_acc, label='Validation Accuracy')
plt.title('ASPECT CATEGORY TRAINING VS VALIDATION ACCURACY')
plt.xlabel('Epochs', fontsize=19, labelpad=20)
plt.ylabel('Accuracy', fontsize=19, labelpad=20)
plt.tick_params(axis='both', which='major', labelsize=15, )
plt.legend(fontsize=17, loc='lower right')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, polarity_acc, label='Training Accuracy')
plt.plot(epochs_range, val_polarity_acc, label='Validation Accuracy')
plt.title('ASPECT SENTIMENT TRAINING VS VALIDATION ACCURACY')
plt.xlabel('Epochs', fontsize=19, labelpad=20)
plt.ylabel('Accuracy', fontsize=19, labelpad=20)
plt.tick_params(axis='both', which='major', labelsize=15)
plt.legend(fontsize=17, loc='lower right')

plt.subplot(2, 2, 3)
plt.plot(epochs_range,category_loss, label='Training Loss')
plt.plot(epochs_range, val_category_loss, label='Validation Loss')
plt.title('ASPECT CATEGORY TRAINING VS VALIDATION LOSS')
plt.xlabel('Epochs', fontsize=19, labelpad=20)
plt.ylabel('MSE', fontsize=19, labelpad=20)
plt.tick_params(axis='both', which='major', labelsize=15)
plt.legend(fontsize=17)

plt.subplot(2, 2, 4)
plt.plot(epochs_range,polarity_loss, label='Training Loss')
plt.plot(epochs_range, val_polarity_loss, label='Validation Loss')
plt.title('ASPECT SENTIMENT TRAINING VS VALIDATION LOSS')
plt.xlabel('Epochs', fontsize=19, labelpad=20)
plt.ylabel('MSE', fontsize=19, labelpad=20)
plt.tick_params(axis='both', which='major', labelsize=15)
plt.legend(fontsize=17)
           
           
plt.show()




------------------------------------------------------------------------------------------------------------------

## VALIDATION 

In [ ]:
categories = y_ohe_train.columns[:-1]
polarity_labels = ['POSITIVE', 'NEGATIVE']

In [ ]:
# ASPECT CATEGORY
y_pred_aspect = multitask_model.predict(test_embeds2)
y_pred_categories = y_pred_aspect[0]
y_pred_polarity = y_pred_aspect[1].reshape(-1)
y_pred_polarity =  [ round(ele) for ele in y_pred_polarity ]

y_pred_aspect_idx = np.argmax(y_pred_categories, axis=1)
y_test_idx =np.argmax(y_ohe_test[aspect_categories].values, axis=1)


In [ ]:
# CONFUSION MATRIX
import seaborn as sns
from sklearn.metrics import confusion_matrix as cm

cm_aspect = cm(y_test_idx, y_pred_aspect_idx)
cm_polarity = cm(y_ohe_test.polarity.values, y_pred_polarity)

In [ ]:
# CLASS-WISE ACCURACY
cm_aspect_acc = cm_aspect.astype('float') / cm_aspect.sum(axis=1)[:, np.newaxis]
aspect_acc =cm_aspect_acc.diagonal()

# POLARITY ACCURACY
cm_polarity_acc = cm_polarity.astype('float') / cm_polarity.sum(axis=1)[:, np.newaxis]
polarity_acc =cm_polarity_acc.diagonal()

# ASPECT CATEGORY
print('*** CLASS-WISE ACCURACY : **** \n')
print("CLASS\tACCURACY")
for cat, acc in zip(aspect_categories, aspect_acc):
    print('',cat,"\t",round(acc, 3))
    
# POLARITY
print('\n\n*** POLARITY ACCURACY : **** \n')
print("CLASS\t\tACCURACY")
for cat, acc in zip(polarity_labels, polarity_acc):
    print(cat,"\t",round(acc, 3))

In [ ]:
plt.figure(figsize=(25, 10))
sns.heatmap(cm_aspect, square=True, annot=True, cmap='Blues', fmt='d', cbar=True, yticklabels=categories, xticklabels=categories)

plt.xlabel('PREDICTED VALUES', fontsize=12, labelpad=20)
plt.ylabel('ACTUAL VALUES', fontsize=12, labelpad=20)
plt.title('ASPECT CATEGORY CONFUSION MATRIX')
plt.show()

sns.heatmap(cm_polarity, square=True, annot=True, cmap='Blues', fmt='d', cbar=True, yticklabels=polarity_labels, xticklabels=polarity_labels)
plt.xlabel('PREDICTED VALUES', fontsize=12, labelpad=20)
plt.ylabel('ACTUAL VALUES', fontsize=12, labelpad=20)
plt.title('ASPECT POLARITY CONFUSION MATRIX')
plt.show()

In [ ]:
from pretty_confusion_matrix import pp_matrix

df_cm = pd.DataFrame(cm_aspect, index=categories, columns=categories)
cmap = 'PuRd'
pp_matrix(df_cm, cmap=cmap)

In [ ]:
df_polarity = pd.DataFrame(cm_polarity, index=polarity_labels, columns=polarity_labels)
cmap = 'PuRd'
pp_matrix(df_polarity, cmap=cmap)

In [ ]:
# CLASSIFICATION REPORT
from sklearn.metrics import classification_report
print(classification_report(y_test_idx, y_pred_aspect_idx, target_names=categories))
print('\n\n',classification_report(y_ohe_test.polarity.values, y_pred_polarity, target_names=polarity_labels))

## **INFERENCE MODEL**

In [ ]:
def predictPreprocess(model, sent):
  outputs = sent2embed(model, sent)
  pred_X = np.array([outputs[1].detach().numpy()[0]])
  embed_X = tf.stack([pred_X])
  return embed_X

def predictPreprocessWithoutBiLSTM(model, sent):
  outputs = sent2embed(model, sent)
  pred_X = np.array([outputs[1].detach().numpy()[0]])
  embed_X = tf.stack(pred_X)
  return embed_X

In [ ]:
def inferenceModel(multi_model, embed_model, sent):
    labels = y_ohe_train.columns[:-1]
#     labels = ['CLA', 'CMP', 'MOT', 'ORI', 'REP', 'SOU', 'SUB', 'SUM']
    sents= sent.split('.')[:-1]
    results =[]
    for s in sents:
#         test_embed = predictPreprocess(embed_model, s.strip())
        test_embed = predictPreprocessWithoutBiLSTM(embed_model, s.strip())
        category_probab = multi_model.predict(test_embed)
        category_label_idx = np.argmax(category_probab[0])
        category_polarity = 1 if category_probab[1][0][0] > 0.5 else 0
        category_label = 'POS' if category_probab[1][0][0] > 0.5 else 'NEG'
#         labels[category_label] , category_polarity
        # print("CATEGORY PROBABILITY : ", category_probab[0][0]) 
        results.append([labels[category_label_idx] , category_polarity, category_label])
        
    return results


In [ ]:
# SAMPLE 1
sent = '''This paper presents a model-free policy gradient approach for reinforcement learning that combines on-policy 
updates with an off-policy critic . The hope is to learn continuous control in a sample-efficient fashion . The approach 
is validated on a number of low-dimensional continuous control tasks in a simulated environment .'''
op = inferenceModel(model,embed_model, sent)

print('PREDICTED OUTPUT :', op)

In [ ]:
# SAMPLE 2
sent = '''However , since the processing of sequential data seems to be a broad and general area of application , it is 
conceivable that this work will be useful in the design and application of future CNNs . All in all , this paper introduces 
an interesting idea to address an important topic .'''
op = inferenceModel(model,embed_model, sent)

print('PREDICTED OUTPUT :', op)